In [19]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Statistics

This notebook contains plots and other code to explore guesses of the players of the game.

### Table of Content:

* ['Guesses' stats](#guess-stats)
* [Ground truth related stats](#ground-truth-stats)


In [20]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
%run ./prepare_data.ipynb






## <a class="anchor" id="guess-stats">Guesses stats</a>

### Most popular and least votes

In [21]:
# trim the long url string for readibility
def trim_url(df, df_idx, str_lst):
    df_simple = df.copy()
    for s in str_lst:
        df_simple[df_idx] = df_simple[df_idx].str.replace(s, '')
    return df_simple

#def highlight_rows(df,v):
#    if df.col > v:
#        return ['background-color: yellow']*df.shape[1]
#    else:
#        return ['background-color: white']*df.shape[1]
#df_guess_simple.style.apply(highlight_rows(0), axis=1)

In [22]:
yt_str = ['^[^_]*=','https://youtu.be/']

#print all votes
if ((df_guess['Current URL']).apply(len).max(axis=0) > 10):
    df_guess_simple = trim_url(df_guess,'Current URL',yt_str)
else:
    df_guess_simple = df_guess
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # printing all, more options can be specified also
    print(df_guess_simple)



   Current URL     Guess  votes
0        URL10   Player4      4
1        URL10   Player6      3
2        URL10   Player3      2
3        URL11   Player1      2
4        URL11   Player9      2
5         URL6   Player6      2
6        URL10   Player1      1
7        URL10   Player2      1
8        URL11  Player10      1
9        URL11  Player11      1
10       URL11   Player2      1
11       URL11   Player3      1
12       URL11   Player5      1
13       URL11   Player7      1
14       URL11   Player8      1
15        URL6   Player3      1


## <a class="anchor" id="ground-truth-stats">Ground truth related stats </a>

First, we compute the intersection between players/url column of ground truth df and guesses/url column of guess df, for all URLs.
Note that one guess is inherently incorrect (by the rules of the game).



In [23]:
# Correct guesses
correct_guess_df = pd.merge(df_raw,df2_raw, how='inner', left_on=['Current URL', 'Guess'], right_on=['URL','Player'])
#del correct_guess_df['Player_y'], correct_guess_df['URL']
correct_guess_df = correct_guess_df.drop(['Player_y', 'URL'], axis = 1)
correct_guess_df.rename(columns = {'Player_x':'Player','Current URL':'URL'}, inplace = True)

correct_guess_simple = trim_url(correct_guess_df, 'URL', yt_str)
print(correct_guess_simple)


     URL   Player     Guess
0  URL11  Player8  Player11
1   URL6  Player9   Player6
2   URL6  Player1   Player6


In [24]:
# Correct guesses per player
#TODO: display this better

corr_guess_stats = correct_guess_df.groupby(by=['Player'])['Guess'].count()
corr_guess_stats = pd.merge(df2_raw, corr_guess_stats, on='Player',how='left').fillna(0)
del corr_guess_stats['URL']
print(corr_guess_stats)

      Player  Guess
0    Player1    1.0
1    Player2    0.0
2    Player3    0.0
3    Player4    0.0
4    Player5    0.0
5    Player6    0.0
6    Player7    0.0
7    Player8    1.0
8    Player9    1.0
9   Player10    0.0
10  Player11    0.0


In [25]:
# Best and worst guessed players
#TODO: display this better

corr_player_stats = correct_guess_df.groupby(by=['URL'])['Guess'].count()
corr_player_stats = pd.merge(df2_raw, corr_player_stats, on='URL',how='left').fillna(0)
del corr_player_stats['URL']
#corr_player_stats['Guess'] = pd.to_numeric(corr_player_stats['Guess'])
corr_player_stats['Guess'] = corr_player_stats['Guess'].values.astype(int)
print(corr_player_stats)

      Player  Guess
0    Player1      0
1    Player2      0
2    Player3      0
3    Player4      0
4    Player5      0
5    Player6      2
6    Player7      0
7    Player8      0
8    Player9      0
9   Player10      0
10  Player11      1


In [26]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # printing all, more options can be specified also
    print(total_votes)


      Player  votes
0    Player6      5
1    Player3      4
2    Player4      4
3    Player1      3
4    Player9      2
5    Player2      2
6    Player7      1
7   Player10      1
8   Player11      1
9    Player8      1
10   Player5      1


In [27]:
wrong_votes = corr_player_stats.copy()
wrong_votes.rename(columns = {'Guess':'votes'}, inplace = True)
wrong_votes["votes"] =  total_votes["votes"] - corr_player_stats["Guess"]
print(wrong_votes)

      Player  votes
0    Player1      5
1    Player2      4
2    Player3      4
3    Player4      3
4    Player5      2
5    Player6      0
6    Player7      1
7    Player8      1
8    Player9      1
9   Player10      1
10  Player11      0
